In [7]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import string
import nltk

nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...


True

In [18]:
import os
import sys

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import NewsDataLoader

if __name__ == "__main__":
    data_directory = "../data"
    loader = NewsDataLoader(data_directory)

    merge_df = loader.load_data()
print(merge_df.columns)

Index(['article_id', 'source_name', 'author', 'title', 'description', 'url',
       'published_at', 'content', 'category', 'article', 'title_sentiment',
       'SourceCommonName', 'location', 'Country', 'GlobalRank', 'TldRank',
       'Domain', 'TLD', 'RefSubNets', 'RefIPs', 'IDN_Domain', 'IDN_TLD',
       'PrevGlobalRank', 'PrevTldRank', 'PrevRefSubNets', 'PrevRefIPs'],
      dtype='object')


In [21]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

merge_df['preprocessed_content'] = merge_df['content'].apply(preprocess_text)


In [22]:
import re
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [ps.stem(word) for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

merge_df['preprocessed_content'] = merge_df['content'].apply(preprocess_text)

# Topic Modeling
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(merge_df['preprocessed_content'])

lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_model.fit(tfidf_matrix)

#  Analyzing Topic Diversity
topic_counts = merge_df.groupby('source_name')['topic'].nunique()

# Analyze Topic Trends
# Plot 2D scatter plot

# Example code for plotting
plt.scatter(merge_df['published_at'], merge_df['topic'], c=merge_df['topic_count'], cmap='viridis')
plt.xlabel('Date')
plt.ylabel('Topics')
plt.title('Topic Trends Over Time')
plt.colorbar(label='Count')
plt.show()


KeyError: 'Column not found: topic'

In [26]:
# Preprocessing
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt

# Define preprocessing function
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [ps.stem(word) for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

# Apply preprocessing to 'title' and 'content' columns and create 'preprocessed_title' and 'preprocessed_content' columns
merge_df['preprocessed_title'] = merge_df['title'].apply(preprocess_text)
merge_df['preprocessed_content'] = merge_df['content'].apply(preprocess_text)

# Topic Modeling
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(merge_df['preprocessed_content'])

lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_model.fit(tfidf_matrix)

# Analyzing Topic Diversity
topic_counts = merge_df.groupby('preprocessed_title')['preprocessed_content'].nunique()

# Analyze Topic Trends
# Ensure that 'published_at' and 'topic_count' columns exist in merge_df
plt.scatter(merge_df['published_at'], merge_df['preprocessed_title'], c=merge_df['topic_counts'], cmap='viridis')
plt.xlabel('Date')
plt.ylabel('Topics')
plt.title('Topic Trends Over Time')
plt.colorbar(label='Count')
plt.show()


KeyError: 'Title'